In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
experiment_id = 'klaus_2018-09-24'

In [ ]:
experiment_base_path = Path('/home/klaus/dev/revolve-simulator/l-system/experiments/')
experiment_path = experiment_base_path / experiment_id
if not experiment_path.exists():
    raise Exception(f'Experiment {str(experiment_path)} does not exist.')

In [ ]:
df = pd.read_csv(experiment_path / 'measures2.txt', sep= ' ')
df.head()

In [ ]:
n_measures = len(set(df['measures'].values))

In [ ]:
df = pd.pivot_table(df, values='value', index=['generation', 'genome'], columns=['measures'], aggfunc='last')
df.head()

In [ ]:
genomes_per_generation_count = df.reset_index().groupby(['generation'])['genome'].count()
all_generations_same_amount = ((genomes_per_generation_count - genomes_per_generation_count.iloc[0]) == 0).all()
if not all_generations_same_amount:
    raise Exception('Not all generations have the same number of genomes')

In [ ]:
generations, genomes = list(zip(*df.index.values))
n_generations = len(set(generations))
n_genomes_per_generation = genomes_per_generation_count.iloc[0]

In [ ]:
data = df.values.reshape((n_generations, n_genomes_per_generation, n_measures))